<a target="_blank" href="../cluster" style="font-size:20px">All Applications (YARN)</a>

# Spark configuration

https://spark.apache.org/docs/latest/configuration.html

https://spark.apache.org/docs/latest/running-on-yarn.html

RAM usage (in MB):
- Driver / Application Master: 512 + 384 = 896
- Executor: 512 + 576 = 1088
- Total: 896 + 1088 * 2 = 3072 (yarn limit)

In [15]:
! cat $SPARK_HOME/conf/spark-defaults.conf

# https://spark.apache.org/docs/latest/running-on-yarn.html
# https://spark.apache.org/docs/latest/configuration.html
spark.default.parallelism 32
spark.master yarn
spark.driver.memory 512m
spark.driver.memoryOverhead 384
spark.executor.instances 2
spark.executor.memory 512m
spark.python.worker.memory 320m
spark.executor.memoryOverhead 576
spark.yarn.max.executor.failures 1000

# Create SparkContext
https://spark.apache.org/docs/latest/rdd-programming-guide.html

In [19]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.wa

In [17]:
import json

# Wikipedia dataset

In [20]:
! ls -lh wiki

total 208M
-rw-rw-r-- 1 jovyan root  61M Mar 16 19:30 categories.jsonl
-rw-rw-r-- 1 jovyan root  387 Oct 13  2022 README.txt
-rw-r--r-- 1 jovyan root 2.9M Apr 19 15:25 sample.jsonl
-rw-rw-r-- 1 jovyan root 144M Mar 16 19:30 wiki.jsonl


In [21]:
! head -n 1 wiki/wiki.jsonl

{"title": "April", "text": "April\n\nApril is the fourth month of the year, and comes between March and May. It is one of four months to have 30 days.\n\nApril always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.\n\nApril's flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.\n\nApril comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.\n\nApril begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other's last days are exactly 35 weeks (245 days) apart.\n\nIn common years, April starts on the same day of the week as October of the previous year, and in leap years, May 

In [9]:
# make a sample
! head -n 1000 wiki/wiki.jsonl > wiki/sample.jsonl

# Copy files to HDFS

In [10]:
! hadoop fs -copyFromLocal wiki /

In [11]:
! hadoop fs -ls -h /wiki

Found 4 items
-rw-r--r--   1 jovyan supergroup        387 2023-04-19 15:26 /wiki/README.txt
-rw-r--r--   1 jovyan supergroup     60.9 M 2023-04-19 15:26 /wiki/categories.jsonl
-rw-r--r--   1 jovyan supergroup      2.9 M 2023-04-19 15:26 /wiki/sample.jsonl
-rw-r--r--   1 jovyan supergroup    143.4 M 2023-04-19 15:26 /wiki/wiki.jsonl


# Create RDD
https://spark.apache.org/docs/latest/rdd-programming-guide.html#resilient-distributed-datasets-rdds

In [12]:
rdd = sc.parallelize(range(5))
rdd

NameError: name 'sc' is not defined

In [10]:
rdd = sc.textFile("hdfs:///wiki/wiki.jsonl")
rdd

hdfs:///wiki/wiki.jsonl MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0

# Actions on RDD
https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions

In [11]:
rdd = sc.parallelize(range(5))

In [12]:
rdd.collect()

[0, 1, 2, 3, 4]

In [13]:
rdd.count()

5

In [14]:
rdd.first()

0

In [15]:
rdd.mean()

2.0

In [16]:
rdd.take(2)

[0, 1]

In [17]:
rdd = sc.parallelize(["one", "two"])

In [18]:
rdd.saveAsTextFile("hdfs:///spark.txt")

In [19]:
! hadoop fs -cat "/spark.txt/*"

one
two


# RDD transformations
https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations

In [20]:
rdd = sc.textFile("hdfs:///wiki/wiki.jsonl")

In [21]:
rdd.sample(False, 0.001).collect()

['{"title": "Grand unification theory", "text": "Grand unification theory\\n\\nGrand unification theory or GUT is a model that tries to describe the universe. It was formed by combining three forces - electromagnetic, weak and strong forces. These are three of the fundamental four forces of nature, which are responsible for all of the pushes and pulls in the universe. If gravity is also combined with these forces, then the GUT will become the proposed Theory of Everything.\\n\\nBecause GUT models have more dimensions than currently known, there has not been a single GUT. Particle accelerators, like the Large Hadron Collider have been built to test these GUTs indirectly, as it is too complex to observe effects on particles without them.\\n\\nGUT was first proposed by researchers from CERN in the 1970s. The discovery of the Higgs Boson and other such recent discoveries will get scientists closer to knowing which GUT is correct, and ultimately know the Theory of Everything. This theory wa

In [22]:
rdd.map(lambda x: json.loads(x)['text'].split()[:3]).take(3)

[['April', 'April', 'is'],
 ['August', 'August', '(Aug.)'],
 ['Art', 'Art', 'is']]

In [23]:
rdd.flatMap(lambda x: json.loads(x)['text'].split()[:3]).take(9)

['April', 'April', 'is', 'August', 'August', '(Aug.)', 'Art', 'Art', 'is']

In [24]:
rdd.filter(lambda x: json.loads(x)['text'][0] == 'B').take(1)

['{"title": "Black pudding", "text": "Black pudding\\n\\nBlack pudding is an English name for zwarte pudding. It is food made by cooking down the blood of any mammal (usually pigs or cattle) with meat, fat or filler until it is thick enough to congeal (become firm or solid) when cooled.\\n\\nIn Great Britain, blood sausage is called \\"Black Pudding\\". The ingredients include pig\'s blood, suet, bread, barley and oatmeal. The most common kind of German \\"blutwurst\\" is made from fatty pork meat, beef blood and filler such as barley. Though already cooked and \\"ready to eat\\" it is usually served warm.\\n\\nOther kinds of blood sausage include \\"boudin noir\\" (France), \\"boudin rouge\\" (Creole and Cajun) and \\"morcilla\\" (Spain).\\n\\nA legend says that blood sausage was invented in a bet between two Bavarian butchers drunk on the alcoholic drink absinthe during the 14th century. Homer\'s \\"Odyssey\\" from Ancient Greece says that \\"As when a man besides a great fire has fi

In [25]:
(
    rdd
    .map(lambda x: json.loads(x)['text'].split())
    .takeOrdered(1, lambda x: len(x))
)

[['Garmisch-Partenkirchen',
  'Olympics',
  'Garmisch-Partenkirchen',
  'Olympics',
  'can',
  'mean:',
  '<templatestyles',
  'src="Dmbox/styles.css"',
  '/>']]

# Emulating MapReduce

In [26]:
rdd = sc.parallelize(["this is text", "some more text"])

In [27]:
(
    rdd
    .flatMap(lambda x: [(w, 1) for w in x.split()])
    .collect()
)

[('this', 1), ('is', 1), ('text', 1), ('some', 1), ('more', 1), ('text', 1)]

In [28]:
(
    rdd
    .flatMap(lambda x: [(w, 1) for w in x.split()])
    .groupByKey()
    .collect()
)

[('this', <pyspark.resultiterable.ResultIterable at 0x7f1119be77b8>),
 ('is', <pyspark.resultiterable.ResultIterable at 0x7f1119be7978>),
 ('more', <pyspark.resultiterable.ResultIterable at 0x7f1119be7860>),
 ('text', <pyspark.resultiterable.ResultIterable at 0x7f1119be74e0>),
 ('some', <pyspark.resultiterable.ResultIterable at 0x7f1119be7048>)]

In [29]:
(
    rdd
    .flatMap(lambda x: [(w, 1) for w in x.split()])
    .groupByKey()
    .map(lambda x: (x[0], list(x[1])))
    .collect()
)

[('this', [1]), ('is', [1]), ('more', [1]), ('text', [1, 1]), ('some', [1])]

In [30]:
(
    rdd
    .flatMap(lambda x: [(w, 1) for w in x.split()])
    .groupByKey()
    .map(lambda x: (x[0], len(x[1])))
    .collect()
)

[('this', 1), ('is', 1), ('more', 1), ('text', 2), ('some', 1)]

In [31]:
(
    rdd
    .flatMap(lambda x: [(w, 1) for w in x.split()])
    .reduceByKey(lambda a, b: a + b)
    .collect()
)

[('this', 1), ('is', 1), ('more', 1), ('text', 2), ('some', 1)]

# Solving WordCount

In [32]:
import sys
import re
import string

def mapper(line):
    text = json.loads(line)['text']
    text = re.sub(f'[^{re.escape(string.printable)}]', ' ', text)  # not printable to space
    words = text.lower().split()
    return [(word, 1) for word in words]

In [33]:
mapper("{\"text\": \"This  is  text\"}")

[('this', 1), ('is', 1), ('text', 1)]

In [34]:
%%time
(
    sc.textFile("hdfs:///wiki/sample.jsonl")
    .flatMap(mapper)
    .reduceByKey(lambda a, b: a + b)
    .takeOrdered(10, lambda x: -x[1])
)

CPU times: user 20.1 ms, sys: 0 ns, total: 20.1 ms
Wall time: 1.41 s


[('the', 34538),
 ('of', 15944),
 ('and', 12966),
 ('in', 12448),
 ('a', 11322),
 ('is', 10870),
 ('to', 9469),
 ('are', 5598),
 ('it', 4330),
 ('that', 4057)]

# Catching an error in the code

In [35]:
import sys
import re
import string

def mapper(line):
    text = json.loads(line)['text123']  # error: missing key!
    text = re.sub(f'[^{re.escape(string.printable)}]', ' ', text)  # not printable to space
    words = text.lower().split()
    return [(word, 1) for word in words]

In [ ]:
%%time
(
    sc.textFile("hdfs:///wiki/sample.jsonl")
    .flatMap(mapper)
    .reduceByKey(lambda a, b: a + b)
    .takeOrdered(10, lambda x: -x[1])
)

# Stopping Spark (and the YARN app)

In [37]:
sc.stop()